In [2]:
import os
from PIL import Image, UnidentifiedImageError
from PIL.TiffTags import TAGS
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
from joblib import Parallel, delayed
import gc
import time

In [3]:
import sys
from pathlib import Path

project_root = Path().resolve().parent
if not project_root in [Path(p).resolve() for p in sys.path]:
    sys.path.append(str(project_root))

from src import PATHS

In [ ]:
df = PATHS.data / pd.read_parquet(PATHS.metadata / "df_filepaths.parquet")[["rvl_image_path"]]
df.rename(columns = {"rvl_image_path": "input_path"}, inplace=True)
df["output_path"] = PATHS.processed_data / "ML_images_100x100" / df["input_path"].apply(lambda x: x.relative_to(PATHS.rvl_cdip_images))
df["output_dir"] = df["output_path"].apply(lambda x:x.parent)
df

## 1. Retrait des marges

In [ ]:
def marges(img):
    """
    Détecte les marges haut, bas, gauche, droite d'une image TIFF en mode L.

    Params:
        image_path (str): chemin vers l'image .tiff
    Returns:
        top, bottom, left, right: tailles des marges en pixels
    """
    # 1. récupérer l'image
    img_np = np.array(img)
    height, width = img_np.shape
    
    # Filtre médian pour réduire le bruit ponctuel
    denoised = cv2.medianBlur(img_np, ksize=5)  # ksize impair, typiquement 3 ou 5

    # Floutage gaussien pour lisser avant seuillage
    blurred = cv2.GaussianBlur(denoised, (5, 5), 0)

    # Binarisation adaptative sur image débruitée
    binary = cv2.adaptiveThreshold(
        blurred, 255,
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV,
        blockSize=25,
        C=15
    )
    
    # 4. Projections
    rows_sum = binary.sum(axis=1)
    cols_sum = binary.sum(axis=0)
    # Filtrer les lignes à analyser
    top_bottom_ignore = 30
    side_ignore=50 
    # On enlève les bords parce qu'il peut y avoir des traits noirs dus à un scan de mauvaise qualité
    valid_rows = rows_sum[top_bottom_ignore:height - top_bottom_ignore]
    valid_cols = cols_sum[side_ignore:width - side_ignore]

    if valid_rows.max() == 0 or valid_cols.max() == 0:
        # Aucun texte détecté
        return 0, height, 0, width

    # Marges haut/bas (avec décalage)
    top = np.argmax(valid_rows > 0) + top_bottom_ignore
    bottom = height - top_bottom_ignore - np.argmax(valid_rows[::-1] > 0)

    # Marges gauche/droite (avec décalage)
    left = np.argmax(valid_cols > 0) + side_ignore
    right = width - side_ignore - np.argmax(valid_cols[::-1] > 0)

    return top, bottom,  left, right

## 2. resize 100 x 100

In [ ]:
# en fait, pas besoin d'une fonction vu qu'elle existe déjà x)

## 3. Test sur 1 image

In [ ]:
#juste pour la visualisation

def add_black_border(image, border_size=1):
    return cv2.copyMakeBorder(
        image,
        top=border_size,
        bottom=border_size,
        left=border_size,
        right=border_size,
        borderType=cv2.BORDER_CONSTANT,
        value=0  # 0 pour noir (mode grayscale)
    )
    

In [ ]:
# 1. on récupère l'image
file_path = df.iloc[5,:]["rvl_image_path"]
img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

# 2. Détection des marges et recadrage
top, bottom, left, right = marges(img) 
cropped = img[top:bottom, left:right]

# 3. resize
img_resized = cv2.resize(cropped, (100, 100))

# 4 : Redimensionner proportionnellement
h, w = cropped.shape
if h > w:
    new_h = 100
    new_w = int(w * 100 / h)
else:
    new_w = 100
    new_h = int(h * 100 / w)
resized = cv2.resize(cropped, (new_w, new_h))

# 5 Créer un fond blanc 100x100
canvas = np.full((100, 100), 255, dtype=np.uint8)

# 6 : Coller au centre
y_offset = 0  # image collée en haut
x_offset = (100 - new_w) // 2  # centrage horizontal
canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

#on rajoute une forbude pour mieux visualiser les bords
canvas_b = add_black_border(canvas)
img_b = add_black_border(img)

# Affichage
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].imshow(img_b, cmap='gray', vmin=0, vmax=255)
axes[0].set_title("Image originale")
axes[0].axis('off')

axes[1].imshow(canvas_b, cmap='gray', vmin=0, vmax=255)
axes[1].set_title("Image padded avec liseret noir")
axes[1].axis('off')

plt.tight_layout()
plt.show()


## 4. Application sur l'ensemble des RVL

In [ ]:
def new_image(row):
    input_path = row["input_path"]
    output_dir = row["output_dir"]
    output_path = row["output_path"]

    try:
        img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return f"Erreur lecture: {input_path}"

        top, bottom, left, right = marges(img)
        cropped = img[top:bottom, left:right]
        img_resized = cv2.resize(cropped, (100, 100))

        os.makedirs(output_dir, exist_ok=True)
        cv2.imwrite(output_path, img_resized)

        return None
    except Exception as e:
        return f"Erreur traitement {input_path}: {e}"

In [ ]:
def new_image(row):
    input_path = row["input_path"]
    output_dir = row["output_dir"]
    output_path = row["output_path"]

    try:
        img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            return f"Erreur lecture: {input_path}"

        # Étape 1 : Rogner
        top, bottom, left, right = marges(img)
        cropped = img[top:bottom, left:right]

        # Étape 2 : Redimensionner proportionnellement
        h, w = cropped.shape
        if h > w:
            new_h = 100
            new_w = int(w * 100 / h)
        else:
            new_w = 100
            new_h = int(h * 100 / w)
        resized = cv2.resize(cropped, (new_w, new_h))

        # Étape 3 : Créer un fond blanc 100x100
        canvas = np.full((100, 100), 255, dtype=np.uint8)

        # Étape 4 : Coller au centre
        y_offset = 0  # image collée en haut
        x_offset = (100 - new_w) // 2  # centrage horizontal
        canvas[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized

        # Sauvegarde
        os.makedirs(output_dir, exist_ok=True)
        cv2.imwrite(output_path, canvas)

        return None
    except Exception as e:
        return f"Erreur traitement {input_path}: {e}"

def parallelisation(df, n_jobs=5):
    
    data = df.to_dict("records")
    results = Parallel(n_jobs=n_jobs, backend='loky')(
        delayed(new_image)(row) for row in data
    )

    erreurs = [r for r in results if r is not None]
    if erreurs:
        print(f"{len(erreurs)} erreurs rencontrées :")
        for e in erreurs[:10]:  # aperçu
            print(e)
    return erreurs

In [ ]:
t0 = time.time()
parallelisation(df, n_jobs=5)
print(f"Durée d'exécution: {time.time() - t0:.2f} secondes pour {len(df)} images")

## 5. Creation du DataFrame de 10 000 colonnes (1 ligne = 1 image, 1 colonne = 1 pixel)

In [ ]:
def images_to_pixel_dataframe(df, include_labels=False, label_column=None):
    records = []
    failed = []
    indices=[]

    for index, row in df.iterrows():
        path = row["output_path"]
        try:
            img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                failed.append(path)
                continue
            flat = img.flatten().astype(np.uint8)  # moins de RAM
            if include_labels and label_column in row:
                flat = np.append(flat, row[label_column])
            records.append(flat)
            indices.append(index)
        except Exception as e:
            failed.append(path)

    n_pixels = 100 * 100
    col_names = [f'p_{i}' for i in range(n_pixels)]
    if include_labels:
        col_names.append(label_column)

    df_pixels = pd.DataFrame(records, columns=col_names, index=indices)

    return df_pixels, failed


# ===============================
#  Traitement par batch
# ===============================
def process_in_batches(df, batch_size, label_column=None):
    output_dir = PATHS.processed_data / "ML_images_100x100"
    os.makedirs(output_dir, exist_ok=True)

    total_batches = len(df) // batch_size + int(len(df) % batch_size != 0)
    all_failed = []

    for i in range(total_batches):
        t0 = time.time()
        df_batch = df.iloc[i * batch_size:(i + 1) * batch_size].copy()
        print(f" Batch {i+1}/{total_batches} - {len(df_batch)} images")

        # Traitement du batch
        df_pixels, failed = images_to_pixel_dataframe(df_batch, include_labels=label_column is not None, label_column=label_column)
        all_failed.extend(failed)

        # Sauvegarde du parquet
        batch_file = output_dir / f"df_pixels_batch_{i:04d}.parquet"
        df_pixels.to_parquet(batch_file)
        print(f" Batch {i+1} terminé en {time.time() - t0:.2f} sec - {len(df_pixels)} images traitées")

        # Nettoyage mémoire
        del df_batch, df_pixels
        gc.collect()

    # Sauvegarde des erreurs
    with open(os.path.join(output_dir, "erreurs_log.txt"), "w") as f:
        for path in all_failed:
            f.write(path + "\n")

    print(f"! Total erreurs: {len(all_failed)}")

# ===============================
#  Concaténation finale
# ===============================
def concat_all_batches():
    output_dir = PATHS.processed_data / "ML_images_100x100"
    parquet_files = sorted([
        os.path.join(output_dir, f) for f in os.listdir(output_dir)
        if f.startswith("df_pixels_batch_") and f.endswith(".parquet")
    ])

    print(f" Concaténation de {len(parquet_files)} fichiers...")
    df_all = pd.concat([pd.read_parquet(f) for f in parquet_files])
    df_all.to_parquet(PATHS.processed_data / "df_pixels.parquet")
    print(f" Fichier final sauvegardé: df_pixels.parquet ({len(df_all)} images)")

# ===============================
#  Exécution
# ===============================
if __name__ == "__main__":
    # Test sur un petit échantillon (ex: 100 images)
    #df_test = df.sample(n=100, random_state=42).copy()
    #process_in_batches(df_test, batch_size=20, processed_data_path=processed_data_path, label_column=None)
    #concat_all_batches(processed_data_path)
    
    # Suppose que `df` et `processed_data_path` sont déjà définis
    batch_size = 5000  # Ajuste si tu veux moins
    process_in_batches(df, batch_size, label_column=None)
    concat_all_batches()

In [ ]:
concat_all_batches(PATHS.processed_data)

## 6. Completion du fichier avec fonction simple pour les données manquantes
A permis la mmise à jour de certaines fonctions. Ne devrait plus être utile maintenant

### 6.1 Identification des données manquantes    

In [ ]:
created = pd.read_parquet(PATHS.processed_data / "df_pixels.parquet")
len(created)

In [ ]:
df_missing = df[~df.index.isin(created.index)]
df_missing.head()

### 6.2 Calcul des pixels manquants

In [ ]:
indices = []
records = []
for index, row in df_missing.iterrows():

    input_path = row['input_path']
    output_path = row['output_path']
    output_dir = row['output_dir']
    img = cv2.imread(input_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Erreur lecture: {input_path}")
    top, bottom, left, right = marges(img)
    cropped = img[top:bottom, left:right]
    img_resized = cv2.resize(cropped, (100, 100))
    flat = img_resized.flatten().astype(np.uint8)  # moins de RAM
    records.append(flat)
    indices.append(index)

n_pixels = 100 * 100
col_names = [f'p_{i}' for i in range(n_pixels)]
df_missing_pixels = pd.DataFrame(records, columns=col_names, index=indices)

df_missing_pixels.head()


### 6.3 Ajout à la df pixels

In [ ]:
pd.concat((created, df_missing_pixels)).sort_index().to_parquet(PATHS.processed_data / "df_pixels.parquet")

## 7. Concatenation avec les features

In [26]:
pixels = pd.read_parquet(PATHS.processed_data / "df_pixels.parquet")
features = pd.read_parquet(PATHS.processed_data / "df_img_features.parquet")

In [27]:
features.drop(columns=['Compression'], inplace=True) # trop peu de variation
features = features[[
    'top_marge', 'bottom_marge', 'left_marge', 'right_marge', 'nb_lignes',
    'nb_colonnes', 'Nettete', 'Bruit', 'Ratio_b', 'Ratio_n', 'Entropie',
    'width']].rename(columns = {
    'top_marge': 'top_marge',
    'bottom_marge': 'bottom_marge',
    'left_marge': 'left_marge',
    'right_marge': 'right_marge',
    'nb_lignes': 'nb_lignes',
    'nb_colonnes': 'nb_colonnes',
    'Nettete': 'sharpness',
    'Bruit': 'noise',
    'Ratio_b': 'ratio_b',
    'Ratio_n': 'ratio_n',
    'Entropie': 'entropy',
    'width': 'width'
    })

In [30]:
df_features_pixels = features.join(pixels)

In [31]:
df_features_pixels.to_parquet(PATHS.processed_data / "df_img_features_pixels.parquet")

## 8. Suppression des fichiers temporaires